Лабораторна робота 10 ІАД
Transfer Learning з використанням TensorFlow
Мета : Навчитися застосовувати transfer learning для класифікації зображень, використовуючи попередньо натреновані моделі у TensorFlow. Ознайомитися з основами fine-tuning моделі для покращення точності на нових даних.
Завдання

1. Вибір моделі: Оберіть одну з доступних у TensorFlow натренованих моделей, таких як MobileNetV2 або ResNet50. Ці моделі мають бути завантажені з TensorFlow Hub.

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2

2. Збір даних: Використайте датасет, що містить зображення для класифікації, наприклад, зображення котів і собак (датасет CIFAR-10 або інший подібний).
Передобробка даних: Завантажте та підготуйте дані, нормалізуючи зображення та масштабуючи їх під вимоги обраної моделі.

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

selected_classes = [3, 5]
train_mask = np.isin(y_train, selected_classes)
test_mask = np.isin(y_test, selected_classes)

x_train, y_train = x_train[train_mask.squeeze()], y_train[train_mask].squeeze()
x_test, y_test = x_test[test_mask.squeeze()], y_test[test_mask].squeeze()

y_train = np.where(y_train == 3, 0, 1)
y_test = np.where(y_test == 3, 0, 1)

x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0


IMG_SIZE = 224  # for MobileNetV2
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

train_generator = train_datagen.flow(
    x_train, y_train,
    batch_size=BATCH_SIZE,
    shuffle=True,
    subset='training'
)

validation_generator = train_datagen.flow(
    x_train, y_train,
    batch_size=BATCH_SIZE,
    shuffle=True,
    subset='validation'
)

test_generator = test_datagen.flow(
    x_test, y_test,
    batch_size=BATCH_SIZE,
    shuffle=False
)

3. Адаптація моделі:
Завантажте обрану модель із TensorFlow Hub і заморозьте всі шари, окрім верхніх кількох.

In [4]:
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False

Додайте новий повнозв’язаний шар (fully connected layer) з кількістю виходів, що дорівнює кількості класів у вашому наборі даних.

In [5]:
inputs = layers.Input(shape=(32, 32, 3))
x = layers.Resizing(IMG_SIZE, IMG_SIZE)(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1, activation='sigmoid')(x)

4. Fine-tuning:
Розморозьте деякі шари в моделі для подальшого донавчання (fine-tuning).
Проведіть донавчання (тренування) моделі на новому датасеті.

In [6]:
model = models.Model(inputs, x)

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=5,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

Epoch 1/5


c:\Users\Назар\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 144s 544ms/step - accuracy: 0.4948 - loss: 0.7029 - val_accuracy: 0.5215 - val_loss: 0.7206
Epoch 2/5


c:\Users\Назар\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 134s 537ms/step - accuracy: 0.4960 - loss: 0.7055 - val_accuracy: 0.4785 - val_loss: 0.7004
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 108us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 120s 479ms/step - accuracy: 0.5040 - loss: 0.6979 - val_accuracy: 0.5215 - val_loss: 0.6980


5. Оцінка моделі: Оцініть точність моделі та побудуйте матрицю похибок для демонстрації її ефективності.
Візуалізація результатів: Покажіть зображення з тестового набору даних із передбаченими класами, щоб продемонструвати роботу моделі.
Додаткове завдання

In [7]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f"Точність на тестових даних (без fine-tuning): {test_accuracy:.2f}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 24s 381ms/step - accuracy: 0.5057 - loss: 0.7028
Точність на тестових даних (без fine-tuning): 0.50


In [8]:
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy', metrics=['accuracy'])

history_fine_tune = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=5,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 514s 2s/step - accuracy: 0.5808 - loss: 0.7068 - val_accuracy: 0.4785 - val_loss: 0.7104
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 644us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 491s 2s/step - accuracy: 0.7516 - loss: 0.5094 - val_accuracy: 0.5215 - val_loss: 0.7863
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 132us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 488s 2s/step - accuracy: 0.7900 - loss: 0.4446 - val_accuracy: 0.5215 - val_loss: 0.9830
